In [ ]:
import safep
import alchemlyb
from glob import glob
import numpy as np
import matplotlib.pyplot as plt
import os
from alchemlyb.parsing import namd
from IPython.display import display, Markdown
from pathlib import Path
from dataclasses import dataclass
import scipy as sp
from alchemlyb.estimators import BAR
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

In [ ]:
import pandas as pd
@dataclass
class FepRun:
    u_nk:           pd.DataFrame
    perWindow:      pd.DataFrame
    cumulative:     pd.DataFrame
    forward:        pd.DataFrame
    forward_error:  pd.DataFrame
    backward:       pd.DataFrame
    backward_error: pd.DataFrame
    per_lambda_convergence: pd.DataFrame
    color: str

# User parameters

In [ ]:
dataroot = Path('.')
replicas = dataroot.glob('./Sample_Data/')
filename_regex='*.fepout'

temperature = 303.15
RT = 0.00198720650096 * temperature
detectEQ = True #Flag for automated equilibrium detection

In [ ]:
colors = ['blue', 'red', 'green', 'purple', 'orange', 'violet', 'cyan']
itcolors = iter(colors)

# Extract key features from the MBAR fitting and get ΔG
Note: alchemlyb operates in units of kT by default. We multiply by RT to convert to units of kcal/mol.

# Read and plot number of samples after detecting EQ

In [ ]:
fepruns = {}
for replica in replicas:
    print(f"Reading {replica}")
    unkpath = replica.joinpath('decorrelated.csv')
    u_nk = None
    if unkpath.is_file():
        print(f"Found existing dataframe. Reading.")
        u_nk = safep.read_UNK(unkpath)
    else:
        print(f"Didn't find existing dataframe at {unkpath}. Checking for raw fepout files.")
        fepoutFiles = list(replica.glob(filename_regex))
        totalSize = 0
        for file in fepoutFiles:
            totalSize += os.path.getsize(file)
        print(f"Will process {len(fepoutFiles)} fepout files.\nTotal size:{np.round(totalSize/10**9, 2)}GB")

        if len(list(fepoutFiles))>0:
            print("Reading fepout files")
            fig, ax = plt.subplots()

            u_nk = namd.extract_u_nk(fepoutFiles, temperature)
            u_nk = u_nk.sort_index(axis=0, level=1).sort_index(axis=1)
            safep.plot_samples(ax, u_nk, color='blue', label='Raw Data')

            if detectEQ:
                print("Detecting equilibrium")
                u_nk = safep.detect_equilibrium_u_nk(u_nk)
                safep.plot_samples(ax, u_nk, color='orange', label='Equilibrium-Detected')

            plt.savefig(f"./{str(replica)}_FEP_number_of_samples.pdf")
            plt.show()
            safep.save_UNK(u_nk, unkpath)
        else:
            print(f"WARNING: no fepout files found for {replica}. Skipping.")
    
    if u_nk is not None:
        fepruns[str(replica)] = FepRun(u_nk, None, None, None, None, None, None, None, next(itcolors))
        

In [ ]:
for key, feprun in fepruns.items():
    u_nk = feprun.u_nk
    feprun.perWindow, feprun.cumulative = safep.do_estimation(u_nk) #Run the BAR estimator on the fep data
    feprun.forward, feprun.forward_error, feprun.backward, feprun.backward_error = safep.do_convergence(u_nk) #Used later in the convergence plot'
    feprun.per_lambda_convergence = safep.do_per_lambda_convergence(u_nk)



# Plot data

In [ ]:
toprint = ""
dGs = []
errors = []
for key, feprun in fepruns.items():
    cumulative = feprun.cumulative
    dG = np.round(cumulative.BAR.f.iloc[-1]*RT, 1)
    error = np.round(cumulative.BAR.errors.iloc[-1]*RT, 1)
    dGs.append(dG)
    errors.append(error)

    changeAndError = f'{key}: \u0394G = {dG}\u00B1{error} kcal/mol'
    toprint += '<font size=5>{}</font><br/>'.format(changeAndError)

toprint += '<font size=5>{}</font><br/>'.format('__________________')
mean = np.average(dGs)

#If there are only a few replicas, the MBAR estimated error will be more reliable, albeit underestimated
if len(dGs)<3:
    sterr = np.sqrt(np.sum(np.square(errors)))
else:
    sterr = np.round(np.std(dGs),1)
toprint += '<font size=5>{}</font><br/>'.format(f'mean: {mean}')
toprint += '<font size=5>{}</font><br/>'.format(f'sterr: {sterr}')
Markdown(toprint)

In [ ]:
dGs

In [ ]:
def do_agg_data(dataax, plotax):
    agg_data = []
    lines = dataax.lines
    for line in lines:
        agg_data.append(line.get_ydata())
    flat = np.array(agg_data).flatten()
    kernel = sp.stats.gaussian_kde(flat)
    pdfX = np.linspace(-1, 1, 1000)
    pdfY = kernel(pdfX)
    std = np.std(flat)
    mean = np.average(flat)
    temp = pd.Series(pdfY, index=pdfX)
    mode = temp.idxmax()

    textstr = r"$\rm mode=$"+f"{np.round(mode,2)}"+"\n"+fr"$\mu$={np.round(mean,2)}"+"\n"+fr"$\sigma$={np.round(std,2)}"
    props = dict(boxstyle='square', facecolor='white', alpha=1)
    plotax.text(0.175, 0.95, textstr, transform=plotax.transAxes, fontsize=14,
            verticalalignment='top', bbox=props)

    return plotax

In [ ]:
fig = None
for key, feprun in fepruns.items():
    if fig is None:
        fig, axes = safep.plot_general(feprun.cumulative, None, feprun.perWindow, None, RT, hysttype='lines', label=key, color=feprun.color)
    else:
        fig, axes = safep.plot_general(feprun.cumulative, None, feprun.perWindow, None, RT, fig=fig, axes=axes, hysttype='lines', label=key, color=feprun.color)
    #fig.suptitle(changeAndError)

# hack to get aggregate data:
axes[3] = do_agg_data(axes[2], axes[3])


axes[0].set_title(str(mean)+r'$\pm$'+str(sterr)+' kcal/mol')
axes[0].legend()
plt.savefig(dataroot.joinpath('FEP_general_figures.pdf'))

# Plot the estimated total change in free energy as a function of simulation time; contiguous subsets starting at t=0 ("Forward") and t=end ("Reverse")

In [ ]:
fig, convAx = plt.subplots(1,1)

for key, feprun in fepruns.items():
    convAx = safep.convergence_plot(convAx, 
                                    feprun.forward*RT, 
                                    feprun.forward_error*RT, 
                                    feprun.backward*RT,
                                    feprun.backward_error*RT,
                                    fwdColor=feprun.color,
                                    bwdColor=feprun.color,
                                    errorbars=False
                                    )
    convAx.get_legend().remove()

forward_line, = convAx.plot([],[],linestyle='-', color='black', label='Forward Time Sampling')
backward_line, = convAx.plot([],[],linestyle='--', color='black', label='Backward Time Sampling')
convAx.legend(handles=[forward_line, backward_line])
ymin = np.min(dGs)-1
ymax = np.max(dGs)+1
convAx.set_ylim((ymin,ymax))
plt.savefig(dataroot.joinpath('FEP_convergence.pdf'))

In [ ]:
genfig = None
for key, feprun in fepruns.items():
    if genfig is None:
        genfig, genaxes = safep.plot_general(feprun.cumulative, None, feprun.perWindow, None, RT, hysttype='lines', label=key, color=feprun.color)
    else:
        genfig, genaxes = safep.plot_general(feprun.cumulative, None, feprun.perWindow, None, RT, fig=genfig, axes=genaxes, hysttype='lines', label=key, color=feprun.color)
plt.delaxes(genaxes[0])
plt.delaxes(genaxes[1])

genaxes[3] = do_agg_data(axes[2], axes[3])
genaxes[2].set_title(str(mean)+r'$\pm$'+str(sterr)+' kcal/mol')

for txt in genfig.texts:
    print(1)
    txt.set_visible(False)
    txt.set_text("")
plt.show()
plt.savefig(dataroot.joinpath('FEP_perLambda_convergence.pdf'))

In [ ]:
from scipy.stats import wasserstein_distance

In [ ]:
states = u_nk.columns
matrices = []
overlap_series = []

bins = np.linspace(-8,8,100)

step = 500
nsteps = 50000
time = np.arange(0, nsteps, step)
for cutoff in time:
    matrix = pd.DataFrame(index=states, columns=states)
    overlap = pd.DataFrame(index=states, columns=states)
    for i in states:
        for j in states:
            fwd_slice = u_nk.loc[(slice(None),i),j]
            mask = fwd_slice.index.get_level_values(0)<cutoff
            fwd = fwd_slice.values[mask]

            bwd_slice = u_nk.loc[(slice(None),j),i]
            mask = bwd_slice.index.get_level_values(0)<cutoff
            bwd = bwd_slice.values[mask]

            if len(fwd)>0 and len(bwd)>0 and not np.isnan(fwd[0]) and not np.isnan(bwd[0]):
                fwd_vals, _ = np.histogram(fwd, bins)
                fwd_vals = fwd_vals/np.sum(fwd_vals)
                bwd_vals, _ = np.histogram(-bwd, bins)
                bwd_vals = bwd_vals/np.sum(bwd_vals)
                overlap.loc[i, j] = np.sum(np.min([fwd_vals, bwd_vals], axis=0))
                matrix.loc[i,j]=wasserstein_distance(fwd, -bwd)

    matrices.append(matrix)
    overlap_series.append(overlap)

In [ ]:
fig, ax = plt.subplots()
distances = np.diag(matrix, k=1)
overlaps = np.diag(overlap, k=1)
windows = (states[1:]+states[:-1])/2
ax.plot(windows, 1/distances)
ax.set_ylabel("Inverse Distance")

ovax = ax.twinx()
ovax.plot(windows, overlaps, color="tab:orange")
ovax.set_ylabel("Overlap", color="tab:orange")
ticks = ovax.get_yticklabels()
ovax.set_yticklabels(ticks, color="tab:orange")

In [ ]:
fig, ax = plt.subplots()
for i,j in zip(states[:-1],states[1:]):
    ax.plot(time, [x.loc[i,j] for x in overlap_series], label=f"{i} ↔ {j}", color=np.array([0.8,1,0.4])*i)
    ax.legend()
ax.set_xlabel("Sampling time (steps)")
ax.set_ylabel("Overlap [0,1]")

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.optimize import curve_fit
from scipy.special import erf

def signif(x, p):
    x = np.asarray(x)
    x_positive = np.where(np.isfinite(x) & (x != 0), np.abs(x), 10**(p-1))
    mags = 10 ** (p - 1 - np.floor(np.log10(x_positive)))
    return np.round(x * mags) / mags

def extract_usable_time_series(xdata, the_data, i, j):
    ydata = np.array([x.loc[i,j] for x in the_data]).flatten()
    mask = np.logical_not(np.isnan(ydata))
    ytofit = ydata[mask]
    xtofit = xdata[mask]

    return xtofit, ytofit

def logistic(x, x0, k):
    return 1/(1+np.exp(-k*(x-x0)))

def hyperbola(x, x0, k):
    return np.tanh((x-x0)*k)

def errorf(x, x0, k):
    return erf((x-x0)*k)

In [ ]:
the_func = errorf
guess = (5000,0.001, 0.66)
guess = (5000, 0.001)

xdata = np.arange(0, nsteps, step)
xtofit, ytofit = extract_usable_time_series(xdata, overlap_series, 0.1, 0.2)
popt, pcov = curve_fit(the_func, xtofit, ytofit, guess)
perr = np.sqrt(np.diag(pcov))
upper = popt+(perr*(1.95,-1.95))
lower = popt-(perr*(1.95,-1.95))

plt.plot(xtofit, ytofit)


project = np.linspace(0,500000,100)
label = np.array([{signif(x,2)} for x in popt])
plt.plot(project, the_func(project, *popt), 'k-', label='\n'.join(label.astype(str)))

label = np.array([{signif(x,2)} for x in upper])
plt.fill_between(project, the_func(project, *upper), the_func(project, *lower), color='gray')
plt.legend()